In [1]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import MNIST

In [2]:
# Define the neural network using standard PyTorch #code on RTX 3060 the blog results are from code for T4
class SimpleNet(nn.Module):
    def __init__(self, input_size=784, hidden_size=500, num_classes=10):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()  # CuDNN-optimized ReLU
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

def train(model, device, train_loader, criterion, optimizer):
    model.train()
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        images = images.view(images.size(0), -1)  # Flatten images

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (batch_idx + 1) % 100 == 0:
            print(f'Batch [{batch_idx + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

def test(model, device, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            images = images.view(images.size(0), -1)  # Flatten images
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the test images: {accuracy:.2f}%')

def main():
    # Hyperparameters
    input_size = 784  # 28x28 images
    hidden_size = 500
    num_classes = 10
    batch_size = 100
    learning_rate = 0.001
    num_epochs = 10

    # Transformations
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])

    # MNIST datasets
    train_dataset = MNIST(root='./data', train=True, transform=transform, download=True)
    test_dataset = MNIST(root='./data', train=False, transform=transform, download=True)

    # Data loaders
    num_workers = 2  # Adjust based on your system
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    # Initialize the network
    model = SimpleNet(input_size, hidden_size, num_classes)

    # Use GPU if available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Start timing for training
    training_start_time = time.time()

    # Train the model
    for epoch in range(num_epochs):
        print(f'\nEpoch [{epoch + 1}/{num_epochs}]')
        train(model, device, train_loader, criterion, optimizer)

    # End timing for training
    training_time = time.time() - training_start_time
    print(f'\nTraining Time: {training_time:.2f} seconds')

    # Start timing for testing
    testing_start_time = time.time()

    # Test the model
    test(model, device, test_loader)

    # End timing for testing
    testing_time = time.time() - testing_start_time
    print(f'Testing Time: {testing_time:.2f} seconds')

if __name__ == '__main__':
    main()

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:04<00:00, 2296603.09it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 136145.76it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 1114977.20it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4755498.94it/s]


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw


Epoch [1/10]
Batch [100/600], Loss: 0.2394
Batch [200/600], Loss: 0.3282
Batch [300/600], Loss: 0.2313
Batch [400/600], Loss: 0.1524
Batch [500/600], Loss: 0.2124
Batch [600/600], Loss: 0.1832

Epoch [2/10]
Batch [100/600], Loss: 0.1433
Batch [200/600], Loss: 0.1802
Batch [300/600], Loss: 0.1784
Batch [400/600], Loss: 0.1114
Batch [500/600], Loss: 0.0864
Batch [600/600], Loss: 0.0945

Epoch [3/10]
Batch [100/600], Loss: 0.0380
Batch [200/600], Loss: 0.0656
Batch [300/600], Loss: 0.0321
Batch [400/600], Loss: 0.0337
Batch [500/600], Loss: 0.0535
Batch [600/600], Loss: 0.0272

Epoch [4/10]
Batch [100/600], Loss: 0.0291
Batch [200/600], Loss: 0.0527
Batch [300/600], Loss: 0.0179
Batch [400/600], Loss: 0.1146
Batch [500/600], Loss: 0.0638
Batch [600/600], Loss: 0.0601

Epoch [5/10]
Batch [100/600], Loss: 0.0529
Batch [200/600], Loss: 0.0499
Batch [300/600], Loss: 0.0100
Batch [400/600], Loss: 0.0345
Batch [500/600]

In [4]:
from torchvision import transforms, datasets
# Define the neural network using standard PyTorch
class SimpleNet(nn.Module):
    def __init__(self, input_size=784, hidden_size=500, num_classes=10):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_classes)
        )

    forward = lambda self, x: self.model(x.view(x.size(0), -1))

train_step = lambda model, device, loader, criterion, optimizer: [
    (optimizer.zero_grad(), criterion(model(images.to(device)), labels.to(device)).backward(), optimizer.step())
    for images, labels in loader
]

test_step = lambda model, device, loader: sum(
    (torch.argmax(model(images.to(device)), dim=1) == labels.to(device)).sum().item()
    for images, labels in loader
)

def main():
    # Hyperparameters
    params = {'input_size': 784, 'hidden_size': 500, 'num_classes': 10, 'batch_size': 100, 'lr': 0.001, 'epochs': 10}
    transform = transforms.ToTensor()
    loaders = {
        name: DataLoader(datasets.MNIST('./data', train=(name == 'train'), download=True, transform=transform),
                         batch_size=params['batch_size'], shuffle=(name == 'train'), num_workers=2)
        for name in ['train', 'test']
    }

    # Initialize the network
    model = SimpleNet(params['input_size'], params['hidden_size'], params['num_classes'])
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Loss and optimizer
    criterion, optimizer = nn.CrossEntropyLoss(), optim.Adam(model.parameters(), lr=params['lr'])

    # Measure training time
    start_time = time.time()
    [train_step(model, device, loaders['train'], criterion, optimizer) for _ in range(params['epochs'])]
    print(f'\nTraining Time: {time.time() - start_time:.2f} seconds')

    # Measure testing time
    start_time = time.time()
    correct = test_step(model, device, loaders['test'])
    total = len(loaders['test'].dataset)
    print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')
    print(f'Testing Time: {time.time() - start_time:.2f} seconds')

if __name__ == '__main__':
    main()


Training Time: 39.96 seconds
Accuracy of the model on the test images: 97.66%
Testing Time: 2.57 seconds
